In [2]:
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.engine.topology import Layer
from keras.layers import Activation, Lambda, Flatten
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple
import pandas as pd


def wave_net_activation(x):
    # type: (Layer) -> Layer
    """The WaveNet activation as described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    
    Params:
        x: The layer we want to apply the activation to

    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, nb_filters, kernel_size, dropout_rate=0):
    # type: (Layer, int, int, str, int, int, float) -> Tuple[Layer, Layer]
    """Defines the residual block for the WaveNet

    Params:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall network
        dilation: The dilation power of 2 we are using for this residual block
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.

    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """
    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding='causal', 
                  name='dilated_conv_%d_tanh_s%d' % (i, s))(x)
    
    x = wave_net_activation(conv)

    x = SpatialDropout1D(dropout_rate, name='spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    # residual
    res_x = keras.layers.add([original_x, x])
    
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        print(f'Updated dilations from {dilations} to {new_dilations} .')
        return new_dilations


def WNX(input_layer,
        nb_filters=64,
        kernel_size=2,
        nb_stacks=1,
        dilations=[1, 2, 4, 8, 16, 32],
        use_skip_connections=True,
        dropout_rate=0.0,
        return_sequences=True):
    """Creates a WaveNet layer.

    Params:
        input_layer: A tensor of shape (batch_size, timesteps, input_dim).
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.

    Returns:
        A WaveNetX layer.
    """
    x = input_layer
    x = Convolution1D(nb_filters, 1, padding='causal', name='initial_conv')(x)
    skip_connections = []
    for s in range(nb_stacks):
        for i in dilations:
            x, skip_out = residual_block(x, s, i, nb_filters, kernel_size, dropout_rate)
            skip_connections.append(skip_out)
    if use_skip_connections:
        x = keras.layers.add(skip_connections)
    x = Activation('relu')(x)

    if not return_sequences:
        output_slice_index = -1
        x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
    return x


def compile_wn_model(num_feat,  # type: int
                     num_classes,  # type: int
                     nb_filters,  # type: int
                     kernel_size,  # type: int
                     dilations,  # type: List[int]
                     nb_stacks,  # type: int
                     max_len,  # type: int
                     use_skip_connections=True,  # type: bool
                     return_sequences=True,
                     regression=True,  # type: bool
                     dropout_rate=0.05,  # type: float
                     nb_outputs=1
                 ):
    # type: (...) -> keras.Model
    """Creates a compiled WaveNet model for a given task (i.e. regression or classification).

    Args:
        num_feat: A tensor of shape (batch_size, timesteps, input_dim).
        num_classes: The size of the final dense layer, how many classes we are predicting.
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        max_len: The maximum sequence length, use None if the sequence length is dynamic.
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        regression: Whether the output should be continuous or discrete.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.

    Returns:
        A compiled keras WaveNet.
    """

    dilations = process_dilations(dilations)

    input_layer = Input(name='input_layer', shape=(max_len, num_feat))

    x = WNX(input_layer, nb_filters, kernel_size, nb_stacks, dilations,
            use_skip_connections, dropout_rate, return_sequences)

    #print('x.shape=', x.shape)

    if regression:
        # regression
        preds = Dense(nb_outputs, activation='linear', name='output_dense')(x)
        
        model = Model(inputs=input_layer, outputs=preds)

        adam = optimizers.Adam(lr=0.002, clipnorm=1.)
        model.compile(adam, loss='mean_squared_error', metrics=['mae', 'mse'])
    else:
        # classification
        x = Dense(num_classes)(x)
        x = Activation('softmax', name='output_softmax')(x)
        output_layer = x
#         print(f'model.x = {input_layer.shape}')
#         print(f'model.y = {output_layer.shape}')
        model = Model(input_layer, output_layer)

        adam = optimizers.Adam(lr=0.002, clipnorm=1.)
        model.compile(adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#         print('Adam with norm clipping.')
        
    model_name = 'D-WNX_C{}_B{}_L{}'.format(2, nb_stacks, '-'.join(map(str, dilations)))
#     print(f'Model name = {model_name}.')
    return model

def make_timeseries_instances(timeseries, window_size):
    """Make input features and prediction targets from a `timeseries` for use in machine learning.

    :return: A tuple of `(X, y, q)`.  `X` are the inputs to a predictor, a 3D ndarray with shape
      ``(timeseries.shape[0] - window_size, window_size, timeseries.shape[1] or 1)``.  For each row of `X`, the
      corresponding row of `y` is the next value in the timeseries.  The `q` or query is the last instance, what you would use
      to predict a hypothetical next (unprovided) value in the `timeseries`.
    :param ndarray timeseries: Either a simple vector, or a matrix of shape ``(timestep, series_num)``, i.e., time is axis 0 (the
      row) and the series is axis 1 (the column).
    :param int window_size: The number of samples to use as input prediction features (also called the lag or lookback).
    """
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0]
    
    X = []
    y = []
    q = []
    
    for start in range(0, timeseries.shape[0] - window_size):
        X.append(np.array(timeseries[start:start + window_size]))
        y.append(np.array(timeseries[start + window_size]))
        q.append([timeseries[-(start + window_size):]])
        
    X = np.atleast_3d(X)
    y = np.asarray(y)
    q = np.atleast_3d(q)
        
    print("X", X.shape)
#     print(X)
#     print()
    
    print("y", y.shape)
#     print(y)
#     print()
    
#     X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
#     y = timeseries[window_size:]
#     print('y', y.shape, y)
    q = np.atleast_3d([timeseries[-window_size:]])
    return X, y, q

def evaluate_timeseries(timeseries, window_size):
    nb_samples, nb_series = timeseries.shape
    
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       # Convert 1D vectors to 2D column vectors
    
    X, y, q = make_timeseries_instances(timeseries, window_size)
    
    #epochs = int(np.sqrt(X.shape[0] * X.shape[1]))
    epochs = X.shape[0] * X.shape[1]
    print("epochs ", epochs)
    
    print('\n\nInput features:', X, '\n\nOutput labels:', y, '\n\nQuery vector:', q, sep='\n')
    test_size = int(0.2 * nb_samples)           
    x_train, x_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
#     model.fit(X_train, y_train, epochs=epochs, batch_size=2, validation_data=(X_test, y_test))
# fit the model to the training data

    model = compile_wn_model(return_sequences=False,
                         num_feat=x_train.shape[2],
                         num_classes=0,
                         nb_filters=8,
                         kernel_size=5,
                         dilations=[1, 2, 4, 8, 16],
                         nb_stacks=8,
                         max_len=x_train.shape[1],
                         use_skip_connections=True,
                         regression=True,
                         dropout_rate=0.005,
                         nb_outputs=nb_series)
    
    print(model.summary())

    model.fit(x_train, y_train, epochs=epochs, batch_size=128, verbose=False)

#     pred = model.predict(x_test)
#     print('\n\nactual', 'predicted', sep='\t')
#     for actual, predicted in zip(y_test, pred.squeeze()):
#         print(actual.squeeze(), predicted, sep='\t')
#     print('next', model.predict(q).squeeze(), sep='\t')
    print()
    score = model.evaluate(x_test, y_test, verbose=0)
    print('(loss, mean_absolute_error, mean_squared_error) = ', score)
    return model

In [3]:
def load_timeseries(snap_fname):
    time_series_len = 128
    timeseries = []
    with open(snap_fname, 'r') as fin:
        for l in fin:
            tokens = l.strip().split('\t')
            if len(tokens) != time_series_len:
                continue
            
            timeseries.append([float(x) for x in tokens])
    # Transposing to make time = the rows (axis=0)
    timeseries = np.asarray(timeseries).T
    return timeseries

In [4]:
def load_m4_timeseries(fname):
    data = pd.read_csv(fname)
    data = data.fillna(0)
    data = data.drop(columns=['V1']).values.T
    return data

In [5]:
#timeseries = load_timeseries("MemePhr.txt")
fname = '/Users/bluebalam/projectsX/libreai/projects/minerva_experimental/minerva/minerva/M4_benchmark/M4-methods/Dataset/Train/Quarterly-train.csv'
timeseries = load_m4_timeseries(fname)

In [6]:
timeseries = timeseries[:,0:15]

In [7]:
timeseries.shape

(866, 15)

In [8]:
timeseries[:10,:4]

array([[7407.41231382, 7552.45461911, 8463.8421932 , 8498.94119397],
       [7528.5660743 , 7541.77457066, 8366.10230911, 8409.92644199],
       [7374.70922497, 7466.56833608, 8269.50219151, 8391.44138144],
       [7395.51484763, 7550.33335403, 8256.98532453, 8292.86031049],
       [7654.00798853, 8067.13152222, 8726.91764696, 8798.52111766],
       [7686.84783506, 8063.70101739, 8733.24359072, 8753.99035458],
       [7578.19074266, 7901.02931239, 8664.26008696, 8740.06255558],
       [7904.37671607, 8155.38731599, 8717.39456788, 8695.54065076],
       [7744.04925353, 8031.01032808, 8662.13972695, 8627.44748783],
       [7889.90901325, 8023.24000549, 8629.10189569, 8525.99342353]])

In [9]:
window_size = 4
#make_timeseries_instances(timeseries[:16,:4], window_size)
#evaluate_timeseries(timeseries[:16,:4], window_size)
model = evaluate_timeseries(timeseries, window_size)

X (862, 4, 15)
y (862, 15)
epochs  3448


Input features:
[[[7407.41231382 7552.45461911 8463.8421932  ...  926.9
   5370.         1527.        ]
  [7528.5660743  7541.77457066 8366.10230911 ...  953.5
   4940.         1576.        ]
  [7374.70922497 7466.56833608 8269.50219151 ...  945.1
   5660.         1641.        ]
  [7395.51484763 7550.33335403 8256.98532453 ...  939.7
   5660.         1599.        ]]

 [[7528.5660743  7541.77457066 8366.10230911 ...  953.5
   4940.         1576.        ]
  [7374.70922497 7466.56833608 8269.50219151 ...  945.1
   5660.         1641.        ]
  [7395.51484763 7550.33335403 8256.98532453 ...  939.7
   5660.         1599.        ]
  [7654.00798853 8067.13152222 8726.91764696 ...  974.1
   5780.         1613.        ]]

 [[7374.70922497 7466.56833608 8269.50219151 ...  945.1
   5660.         1641.        ]
  [7395.51484763 7550.33335403 8256.98532453 ...  939.7
   5660.         1599.        ]
  [7654.00798853 8067.13152222 8726.91764696 ...  974.1
  

In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)